In [1]:
import os
import cv2
import numpy as np
from imutils import paths


In [2]:
from sklearn.preprocessing import LabelBinarizer

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [5]:
datapath=r"C:\Users\18301253\Desktop\Project\Frames"

In [6]:
outputmodel=r"C:\Users\18301253\Desktop\Project\CNNModel"
outputlabelbinarizer=r"C:\Users\18301253\Desktop\Project\videoclassificationbinarizer"

In [7]:
Sports_Label=set(['Biking','PoleVault','Swing','WalkingWithDog'])
print("images are loading...")
pathToImages=list(paths.list_images(datapath))
data=[]
labels=[]
count=0
for images in pathToImages:
    count+=1
    label=images.split(os.path.sep)[-2]
    if label not in Sports_Label:
        continue
    image=cv2.imread(images)
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,(240,240))
    data.append(image)
    labels.append(label)
print(" Total",count,"images loaded...")

images are loading...
 Total 22824 images loaded...


In [8]:
data=np.array(data)
labels=np.array(labels)
#hot enoded values as 0,1,2
lb=LabelBinarizer()
labels=lb.fit_transform(labels)


In [9]:
(X_train,X_test,Y_train,Y_test)=train_test_split(data,labels,test_size=0.25,stratify=labels,random_state=42)

In [10]:
trainingAugmentation=ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
validationAugmentation=ImageDataGenerator()
mean=np.array([123.68,116.779,103.939],dtype="float32")
trainingAugmentation.mean=mean
validationAugmentation.mean=mean


In [11]:
#Neural Network
from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Flatten
from keras.layers.core import Dense
import keras.applications.resnet

from keras.layers import Dropout
from keras.models import Model

 

In [12]:
from tensorflow.keras.applications import ResNet50

In [13]:
baseModel = ResNet50(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(512,activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(len(lb.classes_),activation="softmax")(headModel)
model=Model(inputs=baseModel.input,outputs=headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable=False

In [14]:
from tensorflow.keras.optimizers import SGD

In [15]:
epoch=15
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4/epoch)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [16]:
History=model.fit_generator(
trainingAugmentation.flow(X_train,Y_train,batch_size=32),
    steps_per_epoch=len(X_train)//32,
    validation_data=validationAugmentation.flow(X_test,Y_test),
    validation_steps=len(X_test)//32,epochs=epoch
)

C:\Users\18301253\AppData\Local\Temp/ipykernel_18072/3817000565.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  History=model.fit_generator(


Epoch 1/15
534/534 [==============================] - 1399s 3s/step - loss: 1.0821 - accuracy: 0.5713 - val_loss: 0.7303 - val_accuracy: 0.7446
Epoch 2/15
534/534 [==============================] - 1366s 3s/step - loss: 0.8168 - accuracy: 0.6987 - val_loss: 0.6483 - val_accuracy: 0.7758
Epoch 3/15
534/534 [==============================] - 1354s 3s/step - loss: 0.7449 - accuracy: 0.7264 - val_loss: 0.6033 - val_accuracy: 0.7893
Epoch 4/15
534/534 [==============================] - 1385s 3s/step - loss: 0.7088 - accuracy: 0.7424 - val_loss: 0.5832 - val_accuracy: 0.7988
Epoch 5/15
534/534 [==============================] - 1371s 3s/step - loss: 0.6772 - accuracy: 0.7538 - val_loss: 0.5522 - val_accuracy: 0.8085
Epoch 6/15
534/534 [==============================] - 1382s 3s/step - loss: 0.6554 - accuracy: 0.7641 - val_loss: 0.5441 - val_accuracy: 0.8136
Epoch 7/15
534/534 [==============================] - 1391s 3s/step - loss: 0.6335 - accuracy: 0.7706 - val_loss: 0.5197 - val_accuracy:

In [17]:
import pickle
from tensorflow.keras.models import Model
model.save("CNNModel.h5")
lbinarizer=open(r"C:\Users\18301253\Desktop\Project\videoclassificationbinarizer.pickle","wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()

In [ ]:
model.summary()